In [1]:
import backtrader as bt
import yfinance as yf
import pandas as pd

In [11]:
class LongStraddleSimplified(bt.Strategy):
    params = (
        ('period_length', 30),       # Not used in the provided logic, keep or remove
        ('atr_lookback', 14),
        # ('atr_multiplier', 1.0),     # Not used in the provided logic, keep or remove
        ('stop_loss_percent', 2.0),
        # ('take_profit_percent', 10.0), # Replaced by risk_reward_ratio
        # ('period_comm_info', 30),   # Not used in the provided logic, keep or remove
        ('risk_per_trade', 0.02),    # 1% of portfolio value at risk
        ('risk_reward_ratio', 2.0),  # TP is 2x the risk distance
    )

    def __init__(self):
        self.atr = bt.indicators.ATR(period=self.params.atr_lookback)
        self.atr_sma = bt.indicators.SMA(self.atr, period=20)
        self.close_sma = bt.indicators.SMA(self.datas[0].close, period=20)

        # To prevent placing multiple orders while one is pending
        self.order = None
        # Calculate take profit percent based on stop loss and R:R ratio
        self.take_profit_percent = self.params.stop_loss_percent * self.params.risk_reward_ratio

        # Optional: Track open trade details if needed beyond self.position
        # self.entry_price = None
        # self.stop_loss_price = None
        # self.take_profit_price = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        if order.status in [order.Completed]:
            if order.isbuy():
                print(
                    f'{self.datas[0].datetime.date(0)} BUY EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}, '
                    f'Size: {order.executed.size}'
                )
                # Optional: Store entry details if needed later
                # self.entry_price = order.executed.price
                # self.stop_loss_price = order.executed.price * (1.0 - self.params.stop_loss_percent / 100.0)
                # self.take_profit_price = order.executed.price * (1.0 + self.take_profit_percent / 100.0)

            elif order.issell():
                print(
                    f'{self.datas[0].datetime.date(0)} SELL EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}, '
                    f'Size: {order.executed.size}'
                )

            self.bar_executed = len(self) # Bar number when order was executed

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(f'{self.datas[0].datetime.date(0)} Order Canceled/Margin/Rejected')

        # Reset order status tracking
        self.order = None

    def notify_trade(self, trade):
         if not trade.isclosed:
             return
         print(f'{self.datas[0].datetime.date(0)} TRADE PROFIT, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}')


    def next(self):
        # Wait for indicators to have enough data
        # Use max period from all indicators used in the condition + 1
        if len(self) <= max(self.params.atr_lookback, 20) + 1:
             return

        # Check if an order is pending. If so, do not send another one
        # if self.order:
        #     return

        # Check if we are already in the market
        if self.position:
            return
    
        # Calculate current indicator values
        atr_value = self.atr[0]
        atr_sma_value = self.atr_sma[0]
        close_value = self.datas[0].close[0]
        close_sma_value = self.close_sma[0]

        # Define entry condition
        high_volatility_condition = close_value > close_sma_value and atr_value > atr_sma_value

        if high_volatility_condition:
            # Calculate Stop Loss Price based on current close (potential entry)
            potential_entry = close_value
            stop_loss_price = potential_entry * (1.0 - self.params.stop_loss_percent / 100.0)
            take_profit_price = potential_entry * (1.0 + self.take_profit_percent / 100.0)

            print(f"{self.datas[0].datetime.date(0)} --- ENTRY SIGNAL ---")
            print(f"Potential Entry: {potential_entry:.2f}")
            print(f"Stop Loss Price: {stop_loss_price:.2f}")
            print(f"Take Profit Price: {take_profit_price:.2f}")

            # Place Buy order with attached Stop Loss and Take Profit (Bracket Order)
            self.order = self.buy_bracket(
                exectype=bt.Order.Market,
                transmit=True, # Transmit the main order immediately
                # The following create virtual child orders managed by backtrader
                stopprice=stop_loss_price,
                stopexec=bt.Order.Stop, # Stop Market order for SL
                limitprice=take_profit_price,
                limitexec=bt.Order.Limit # Limit order for TP
            )
            # The old way using parent/transmit=False is more complex
            # self.mainside = self.buy(size=size, exectype=bt.Order.Market)
            # self.sell(exectype=bt.Order.Stop, price=stop_loss_price, size=size, parent=self.mainside)
            # self.sell(exectype=bt.Order.Limit, price=take_profit_price, size=size, parent=self.mainside)


In [12]:
# Fetch S&P500 data from Yahoo Finance
# ticker = "^GSPC"  # S&P 500 ticker
# ticker = "AAPL"  # Apple ticker

# ticker = "USDJPY=X"  # USD/JPY ticker
# ticker = "^DJI"  # Dow Jones Industrial Average ticker
ticker = "^IXIC"  # NASDAQ Composite ticker
# Download Tesla data
# ticker = "TSLA"  # Tesla ticker
# Download data from Yahoo Finance
data = yf.download(ticker, start="2018-01-01", end="2023-10-01", interval="1d")

# Convert to Backtrader data feed
data['Date'] = pd.to_datetime(data.index)
data.set_index('Date', inplace=True)

# Clean columns to match Backtrader's expectations
data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data.columns = ['open', 'high', 'low', 'close', 'volume']
data['openinterest'] = 0  # Backtrader requires this column
data['close'] = data['close'].astype(float)
data['open'] = data['open'].astype(float)
data['high'] = data['high'].astype(float)
data['low'] = data['low'].astype(float)
data['volume'] = data['volume'].astype(float)
data['openinterest'] = data['openinterest'].astype(float)
data['datetime'] = data.index
data['datetime'] = pd.to_datetime(data['datetime'])
# data['datetime'] = data['datetime'].apply(lambda x: x.timestamp())
# data['datetime'] = pd.to_datetime(data['datetime'], unit='s')
# data['datetime'] = data['datetime'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')

data_feed = bt.feeds.PandasData(dataname=data,
                                # datetime=0, open=1, high=2, low=3, close=4, volume=5, openinterest=-1
                                )

[*********************100%***********************]  1 of 1 completed


In [13]:
cerebro = bt.Cerebro()

# Add Strategy
cerebro.addstrategy(LongStraddleSimplified)

cerebro.adddata(data_feed)
cerebro.addsizer(bt.sizers.PercentSizer, percents=2)

# Set initial capital
cerebro.broker.setcash(100000.0)

# Set commission - Simplified for example, adjust as needed
cerebro.broker.setcommission(commission=0.001)

# Analyzer for performance metrics
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

In [14]:
# Run Backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value:   %.2f' % cerebro.broker.getvalue())


Starting Portfolio Value: 100000.00
2018-02-20 --- ENTRY SIGNAL ---
Potential Entry: 7234.31
Stop Loss Price: 7089.62
Take Profit Price: 7523.68
2018-02-21 BUY EXECUTED, Price: 7258.48, Cost: 2006.68, Comm: 2.01, Size: 0.27646036509372013
2018-03-02 SELL EXECUTED, Price: 7089.62, Cost: 2006.68, Comm: 1.96, Size: -0.27646036509372013
2018-03-02 Order Canceled/Margin/Rejected
2018-03-02 TRADE PROFIT, GROSS -46.68, NET -50.65
2018-03-02 --- ENTRY SIGNAL ---
Potential Entry: 7257.87
Stop Loss Price: 7112.71
Take Profit Price: 7548.18
2018-03-05 BUY EXECUTED, Price: 7222.89, Cost: 1989.35, Comm: 1.99, Size: 0.27542336712765575
2018-03-09 SELL EXECUTED, Price: 7548.18, Cost: 1989.35, Comm: 2.08, Size: -0.27542336712765575
2018-03-09 Order Canceled/Margin/Rejected
2018-03-09 TRADE PROFIT, GROSS 89.59, NET 85.53
2018-04-12 --- ENTRY SIGNAL ---
Potential Entry: 7140.25
Stop Loss Price: 6997.44
Take Profit Price: 7425.86
2018-04-13 BUY EXECUTED, Price: 7179.62, Cost: 2011.73, Comm: 2.01, Size: 0

In [6]:
# Extract Analyzers
returns_analyzer = results[0].analyzers.returns.get_analysis()
sharpe_analyzer = results[0].analyzers.sharpe.get_analysis()
drawdown_analyzer = results[0].analyzers.drawdown.get_analysis()

print('Returns:', returns_analyzer['rtot'])
print('Sharpe Ratio:', sharpe_analyzer['sharperatio'])
print('Drawdown:', drawdown_analyzer['drawdown'])
print('Max Drawdown:', drawdown_analyzer['max']['drawdown'])
# Optional: Plot the backtest result
cerebro.plot()

Returns: 0.016325308153431087
Sharpe Ratio: 0.32820474882492107
Drawdown: 0.7660094269858404
Max Drawdown: 1.5773378529087687


<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]